# Build Judical RAG system 

### Load the VectorDB

In [23]:
from pymilvus import MilvusClient, Collection, connections

import pandas as pd
import tqdm.autonotebook as tqdm
import numpy as np
import json
import re
from typing import List, Optional




from FlagEmbedding import BGEM3FlagModel


In [3]:
# declare client
client = MilvusClient(uri="http://localhost:19530")

# Step 1: Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# load the collection of law articles
collection_name = "RSchunk_articles_collection"
collection = Collection(collection_name)

# activate the collection
collection = collection.load()


### Load the Embeddings model

In [4]:
bge_m3 = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True, 
                       device='cuda')

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\mthia\anaconda3\envs\AI_App\Lib\site-packages\FlagEmbedding\BGE_M3\modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = torch.load(os

In [18]:
def generate_embedding(query):
    embedding = bge_m3.encode([query], batch_size=12, max_length=1024)["dense_vecs"]
    return embedding[0]

### Load the LLM model

In [5]:
import torch
from my_token import token_key
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

In [6]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=quantization_config,
    device_map="cuda",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##### Generation: Multi Queries

In [8]:
question = "comment payer mes impots en belgique ?"


In [9]:
prompt_multiQuery = f"""À partir de la question de base '{question}', 
génère 3 nouvelles questions connexes qui explorent différents aspects ou angles du sujet.
Assure-toi que chaque nouvelle question apporte une perspective unique ou approfondit un élément spécifique de la question initiale.
Donne uniquement les questions, sans explications ni commentaires supplémentaires.
Format de réponse attendu :
1. [Question 1]
2. [Question 2]
3. [Question 3]

"""

In [135]:
prompt_hyde = f"""Génère une réponse hypothétique concise mais informative à la question suivante :

Question : {question}

Fournissez seulement une réponse qui pourrait plausiblement apparaître dans un document pertinent. La réponse doit être factuelle, directe et informative.

Réponse hypothétique :
"""

In [66]:
prompt_multiQuery

"À partir de la question de base 'Peut-on saisir tous mes revenus si je n'arrive plus à payer mes dettes?', \ngénère 3 nouvelles questions connexes qui explorent différents aspects ou angles du sujet.\nAssure-toi que chaque nouvelle question apporte une perspective unique ou approfondit un élément spécifique de la question initiale.\nDonne uniquement les questions, sans explications ni commentaires supplémentaires.\nFormat de réponse attendu :\n1. [Question 1]\n2. [Question 2]\n3. [Question 3]\n\n"

In [10]:
# Tokenisation du prompt, envoi sur GPU
input_ids = tokenizer(prompt_multiQuery, return_tensors="pt").to(0)

# Génération de nouvelles questions
out = model.generate(
    **input_ids,
    max_new_tokens=250,  # Ajuste selon tes besoins
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True  # Active le sampling pour plus de diversité
)

# Décodage uniquement de la partie générée (ignore le prompt d'entrée)
generated_tokens = out[0][input_ids['input_ids'].shape[-1]:]  # On découpe pour ignorer le prompt
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

# Affichage des questions générées
print(generated_text)
print(len(generated_text.split()))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


1. Quelle est la durée d'attente pour les impôts en Belgique?
2. Quels sont les documents nécessaires pour payer les impôts en Belgique?
3. Quels sont les avantages fiscaux pour les étudiants en Belgique?
34


In [146]:
hyde_response = generated_text.split("Réponse hypothétique :")[-1].strip()
# Ajouter une condition pour s'arrêter après "\n\n" 
hyde_response = hyde_response.split("\n\n")[0].strip()
hyde_response

"Si vous ne payez pas vos impots en Belgique, vous risquez de faire l'objet d'une procédure de recouvrement et de sanctions. Vous serez tenu de payer des pénalités et des intérêts, ainsi que des frais de recouvrement. En cas de défaut de paiement répété, vous pourriez être soumis à des mesures de confiscation de biens ou de rétention de vos revenus. Il est recommandé de s'assurer de vos obligations fiscales et de les honorer régulièrement pour éviter ces conséquences. Il est également possible de contacter l'administration fiscale belge ou d'obtenir de l'aide auprès d'un professionnel pour vous aider à gérer vos impôts."

In [ ]:
def process_questions_and_get_embeddings(base_question, generated_text, bge_m3, method='standard'):
    if method == 'hyde':
        # Extraire la réponse hypothétique
        hyde_response = generated_text.split("Réponse hypothétique :")[-1].strip()
        # Ajouter une condition pour s'arrêter après "\n\n" 
        hyde_response = hyde_response.split("\n\n")[0].strip()
        
        # Combiner la question de base avec la réponse hypothétique
        combined_text = f"Question : {base_question}\nRéponse : {hyde_response}"
        
        # Créer l'embedding pour la combinaison question-réponse
        embedding = bge_m3.encode([combined_text], batch_size=12, max_length=1024)["dense_vecs"][0]
        
        return [embedding], combined_text  # Retourner une liste avec un seul embedding
    
    else:  # méthode standard (multi-query)
        # Extraire les nouvelles questions
        new_questions = re.findall(r'\d+\.\s*(.*)', generated_text)
        # print(new_questions)
        
        # Combiner la question de base avec les nouvelles questions
        all_questions = [base_question] + new_questions
        
        # Créer les embeddings pour chaque question
        embeddings = []
        for question in all_questions:
            embedding = bge_m3.encode([question], batch_size=12, max_length=1024)["dense_vecs"][0]
            embeddings.append(embedding)
        
        return embeddings



In [12]:
# Obtenir les embeddings pour la méthode standard
embeddings_standard = process_questions_and_get_embeddings(question, generated_text, bge_m3, method='standard')

# Obtenir les embeddings pour la méthode HyDE
# embeddings_hyde, combined_text = process_questions_and_get_embeddings(question, generated_text, bge_m3, method='hyde')

### Search

In [13]:
res = client.list_partitions(collection_name=collection_name)
print(res)
selected_partitions = res[1:]

['_default', 'Code_Judiciaire', 'Code_de_la_Dmocratie_Locale_et_de_la_Dcentralisation', 'Code_de_Droit_Economique', 'La_Constitution', 'Code_Civil', 'Code_de_la_Navigation', 'Code_Pnal_Militaire', 'Code_dInstruction_Criminelle', 'Code_Wallon_de_lAgriculture', 'Code_du_Bien_tre_au_Travail', 'Code_Rglementaire_Wallon_de_lAction_sociale_et_de_la_Sant', 'Code_Bruxellois_de_lAmnagement_du_Territoire', 'Code_Wallon_de_lAction_sociale_et_de_la_Sant', 'Code_de_la_Fonction_Publique_Wallonne', 'Code_Wallon_de_lEnseignement_Fondamental_et_de_lEnseignement_Secondaire', 'Code_des_Socits_et_des_Associations', 'Code_Wallon_du_Dveloppement_Territorial', 'Code_Pnal', 'Code_de_lEau_intgr_au_Code_Wallon_de_lEnvironnement', 'Code_Wallon_de_lEnvironnement', 'Code_Wallon_de_lHabitation_Durable', 'Code_Electoral', 'Code_Electoral_Communal_Bruxellois', 'Code_Consulaire', 'Code_Forestier', 'Code_Bruxellois_du_Logement', 'Code_Rural', 'Code_Ferroviaire', 'Codes_des_Droits_et_Taxes_Divers', 'Code_de_Droit_Intern

In [28]:
client.load_partitions(collection_name = collection_name,
                                    partition_names= selected_partitions)

search_results = client.search(
    collection_name= collection_name,
    data = embeddings_standard,  
    partition_names= selected_partitions,
    limit=20,  
    search_params={"metric_type": "COSINE", "params":{}},
    output_fields= ['reference', 'article_id', 'chunk_article']
)

In [29]:
formatted_result = json.dumps(search_results[0], indent=3, ensure_ascii=False)
print(formatted_result)

[
   {
      "id": 21444,
      "distance": 0.6560348272323608,
      "entity": {
         "reference": "Art. 201/9/2, Codes des Droits et Taxes Divers (Livre II, Titre X)",
         "article_id": 22351,
         "chunk_article": "§ 1er Les intermédiaires belges déposent une déclaration au bureau compétent, au plus tard le vingtième jour du troisième mois qui suit la fin de la période de référence.La taxe est payée le jour visé à l'alinéa 1er.Le Roi détermine les modalités de la déclaration 2 L'absence de déclaration, la déclaration tardive, inexacte ou incomplète ainsi que l'absence de paiement ou le paiement tardif sont punis d'une amende qui est établie en fonction de la nature et de la gravité de l'infraction, selon une échelle déterminée par le Roi et allant de 10 p.c à 200 p.c de la taxe due.En l'absence de mauvaise foi, il n'est pas dû d'amende"
      }
   },
   {
      "id": 1692,
      "distance": 0.6369006037712097,
      "entity": {
         "reference": "Art. 201/9, Codes d

In [17]:
predicted_ids = [result["entity"]["article_id"] for result in search_results[0]]
print(predicted_ids)

[22351, 22349, 22289, 22288, 22349, 22366, 22366, 22350, 22319, 22287, 17687, 22316, 5553, 22413, 22309, 22344, 22421, 11941, 22285, 7283]


In [ ]:
5853,5854,5855

### Reranking 

In [31]:
def rerank_documents(question, search_results, bge_m3, max_length=1024, w_d=0.4, w_s=0.2, w_c=0.4):

    # Extraire les textes des articles
    articles = [result["entity"]["chunk_article"] for result in search_results[0]]
    references = [result["entity"]["reference"] for result in search_results[0]]

    
    # Créer les paires question-article
    sentence_pairs = [[question, article] for article in articles]

    # Calculer les scores
    scores = bge_m3.compute_score(
        sentence_pairs,
        max_passage_length=max_length,
        weights_for_different_modes=[w_d, w_s, w_c]  # [dense, sparse, colbert]
    )
    
    print(len(articles))


# Combiner les scores avec les résultats originaux
    ranked_results = [
        (article, reference, score)
        for article, reference, score in zip(articles, references, scores['colbert+sparse+dense'])
    ]

    # Trier les résultats par le nouveau score de reranking (du plus élevé au plus bas)
    ranked_results.sort(key=lambda x: x[-1], reverse=True)


    # Retourner les 3 meilleurs résultats (seulement les articles)
    top_3_articles = [ (article, reference) for article, reference, _ in ranked_results[:3]]

    return top_3_articles


In [32]:
max_length = 1024  
w_d, w_s, w_c = 0.4, 0.2, 0.4  


# Supposons que search_results contient vos 10 résultats initiaux
reranked_results = rerank_documents(question, search_results, bge_m3, max_length, w_d, w_s, w_c)
print(reranked_results)


20
[("§ 1er Les intermédiaires belges déposent une déclaration au bureau compétent, au plus tard le vingtième jour du troisième mois qui suit la fin de la période de référence.La taxe est payée le jour visé à l'alinéa 1er.Le Roi détermine les modalités de la déclaration 2 L'absence de déclaration, la déclaration tardive, inexacte ou incomplète ainsi que l'absence de paiement ou le paiement tardif sont punis d'une amende qui est établie en fonction de la nature et de la gravité de l'infraction, selon une échelle déterminée par le Roi et allant de 10 p.c à 200 p.c de la taxe due.En l'absence de mauvaise foi, il n'est pas dû d'amende", 'Art. 201/9/2, Codes des Droits et Taxes Divers (Livre II, Titre X)'), ("§ 1er L'intermédiaire belge effectue la retenue, la déclaration et le paiement de la taxe", 'Art. 201/9, Codes des Droits et Taxes Divers (Livre II, Titre X)'), ("Les intermédiaires non constitués ou non établis en Belgique peuvent, lorsqu'ils gèrent pour une personne physique un compt

# GENERATIVE

In [33]:
def generate_legal_response(model, tokenizer, prompt_template, documents, question, 
                          max_new_tokens=400, temperature=0.7, top_p=0.9):
    """
    Génère une réponse juridique basée sur les documents fournis.
    
    Args:
        model: Le modèle LLM chargé
        tokenizer: Le tokenizer associé
        prompt_template: Le template du prompt avec {documents} et {question} comme placeholders
        documents: Liste de tuples (contenu, référence)
        question: Question de l'utilisateur
        max_new_tokens: Nombre maximum de tokens à générer
        temperature: Température pour le sampling
        top_p: Valeur pour le nucleus sampling
    
    Returns:
        str: Réponse générée
    """
    # Formatage des documents avec leurs références
    formatted_docs = []
    for i, (content, reference) in enumerate(documents, 1):
        formatted_doc = f"Document {i}:\nRéférence: {reference}\nContenu: {content.strip()}"
        formatted_docs.append(formatted_doc)
    
    formatted_docs = "\n\n".join(formatted_docs)
    
    # Construction du prompt final
    prompt = prompt_template.format(
        documents=formatted_docs,
        question=question
    )
    
    # Tokenisation et envoi sur GPU
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048
    ).to("cuda")
    
    # Génération
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Décodage uniquement de la partie générée
    generated_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return response.strip()


In [34]:
# Template du prompt adapté
prompt_GEN = """[CONTEXTE]
Vous êtes un assistant juridique spécialisé dans le droit belge. Votre rôle est d'aider à comprendre et interpréter la législation belge en vous basant uniquement sur les documents fournis.

[INSTRUCTIONS]
1. Analysez attentivement les extraits de documents juridiques ci-dessous
2. Répondez à la question en utilisant UNIQUEMENT les informations présentes dans ces documents
3. Si certains aspects de la question ne peuvent pas être traités avec les documents fournis, indiquez-le clairement
4. Privilégiez une réponse concise et directe
5. Dans votre réponse, citez systématiquement les références juridiques exactes utilisées (Articles, Codes)

[DOCUMENTS]
{documents}

[QUESTION]
{question}

[FORMAT DE RÉPONSE]
- Réponse directe à la question
- Citation systématique des références juridiques utilisées
- Si des informations manquent, le mentionner brièvement

[TONALITÉ]
- Professionnel mais accessible
- Factuel
- Précis

[RÉPONSE]
"""

response = generate_legal_response(
    model=model,
    tokenizer=tokenizer,
    prompt_template=prompt_GEN,
    documents= reranked_results ,
    question=question
)
print(response)

Pour payer ses impôts en Belgique, il faut déposer une déclaration au bureau compétent au plus tard le vingtième jour du troisième mois qui suit la fin de la période de référence. La taxe est payée le jour visé.

La déclaration doit être faite par l'intermédiaire belge, qui effectue la retenue, la déclaration et le paiement de la taxe, conformément à l'article 201/9 du Code des Droits et Taxes Divers (Livre II, Titre X).

En cas de décès du représentant responsable ou d'un évènement entraînant son incapacité, il est pourvu immédiatement à son remplacement, selon l'article 158/2 du Code des Droits et Taxes Divers (Livre II, Titre II).

Références juridiques utilisées :

- Art. 201/9, Codes des Droits et Taxes Divers (Livre II, Titre X)
- Art. 158/2, Codes des Droits et Taxes Divers (Livre II, Titre II)

Mention: Il est possible que d'autres dispositions ou réglementations applicables soient plus précises ou complètes, mais elles n'ont pas été mises en avant dans les documents fournis. I